# Assignment 3

by Nicolás Larrañaga C and Jonathan Alberto Ortiz Rodriguez

In [2]:
import numpy as np
import math

## 1. a


We start by defining an expression that allows us to compute the average distance to the center of mass in the feature space from a set of input data. 

let's redefine the distance as following

$ || \phi(x) - \phi(y) || = \sqrt{ (x1 - x2)^2 + (y1 - y2)^2 } = \sqrt{ -2<x,y> + <x,x> + <y,y>} $

now we can use this distance formula to calculate the average distance to the center of mass as following

$ \frac{1}{n} \sum_{i = 1}^n || \phi(x_i) - \frac{1}{n} \sum_{j = 1} ^ n \phi(x_j) ||  = $

$ \frac{1}{n} \sum_{i = 1}^ n \sqrt{ <\phi(x_i), \phi(x_i)> + \frac{1}{n^2} < \sum_{j = 1}^n \phi(x_j) , \sum_{k =1}^n\phi(x_k) ] - \frac{2}{n} \phi(x_i)\sum_{j = 1}^n \phi(x_j)  }  = $

$ \frac{1}{n} \sum_{i = 1}^n \sqrt{ k(x_i, x_i)  + \frac{1}{n^2} \sum_{j = 1}^n \sum_{k = 1}^n k(x_j,x_k)  - \frac{2}{n} \sum_{j = 1}^n k(x_i,x_j) } $

this expression can be implemented as follows

In [4]:
def average_distance(x, kernel_mat):
    ans = 0.0
    n = x.shape[0]
    for i in range(n):
        tmp1 = kernel_mat[i][i]
        tmp2 = 0.0
        for j in range(n):
            for k in range(n):
                tmp2 += kernel_mat[j][k]
        tmp2 /= (n**2)
        tmp3 = 0.0
        for j in range(n):
            tmp3 += kernel_mat[i][j]
        tmp3 *= 2.0
        tmp3 /= n
        ans += math.sqrt(tmp1 + tmp2 - tmp3)
    return ans/n
        

## 1.b

In [5]:
x = np.array([(0, 1), (-1, 3), (2, 4), (3, -1), (-1, -2)])
n = x.shape[0]
k_1 = [[x[i].dot(x[j]) for i in range(n)] for j in range(n)] # <x,y>
k_2 = [[x[i].dot(x[j])**2 for i in range(n)] for j in range(n)] #<x,y>^2
k_3 = [[(x[i].dot(x[j]) + 1)**5 for i in range(n)] for j in range(n)] #(<x,y> + 1)^5
k_4 = [[math.exp(- np.linalg.norm(x[i] - x[j]) ** 2 / 2.0) for i in range(n)] for j in range(n)] #gaussian kernel with sigma = 1

### i

In [6]:
average_distance( x, k_1 )

2.5991877273570343

### ii

In [7]:
average_distance(x, k_2)

7.93374823939363

### iii

In [8]:
average_distance(x, k_3)

699.7672616263939

### iV

In [9]:
average_distance(x, k_4)

0.8899495533798308

## 2.a

We start by importing the data in the following way

In [10]:
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing, svm
import matplotlib.pyplot as plt
import numpy as np

mnist = fetch_mldata('MNIST original')
print(mnist)

{'DESCR': 'mldata.org dataset: mnist-original', 'COL_NAMES': ['label', 'data'], 'target': array([0., 0., 0., ..., 9., 9., 9.]), 'data': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}


from this we normalize the data with mean 0 and standard deviation 1

In [11]:
data = np.array(mnist.data).astype(float)
scaled_data = preprocessing.scale(data,copy=False)
print(scaled_data)
print(scaled_data.mean())
print(scaled_data.std())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
-7.739646306420399e-18
0.9576491198422709


## 2.c
now let's define a function that plots the error based on the parameter C that the SVM uses

In [13]:
labels = np.array(mnist.target)
tuples = list(zip(labels.astype(int), scaled_data))

d = {}
for key, val in tuples:
    d.setdefault(key, []).append(val)
    
def test_svm(a, b, training_data_percentage=0.5):
    p_train = training_data_percentage
    p_test = 1.0 - p_train

    # getting samples for each class for training and testing
    a_train_samples = int(len(d[a]) * p_train)
    b_train_samples = int(len(d[b]) * p_train)

    a_test_samples = len(d[a]) - a_train_samples
    b_test_samples = len(d[b]) - b_train_samples
    
    # getting all images of class 1 ,7 for training 
    a_train = d[a][:a_train_samples]
    b_train = d[b][:b_train_samples]
    # getting all images of class 1,7 for  testing
    a_test = d[a][a_train_samples + 1 :]
    b_test = d[b][b_test_samples + 1:]

    #training_data = preprocessing.scale(ones_train + sevens_train)
    #test_data = preprocessing.scale(ones_test + sevens_test)
    training_data = a_train + b_train
    test_data = a_test + b_test
    training_target = [a] * len(a_train) + [b] * len(b_train)
    test_target = [a] * len(a_test) + [b] * len(b_test)
    C = np.logspace(-50, 50, 100)
    train_error = []
    test_error = []
    for c in C:
        clf = svm.SVC(C=c, kernel='linear')
        clf.fit(training_data, training_target)
        train_error.append(clf.score(training_data, training_target))
        test_error.append(clf.score(test_data, test_target))
    
    plt.subplot(2, 1, 1)
    plt.semilogx(C, train_error, label='Train')
    plt.semilogx(C, test_error, label='Test')
    plt.legend(loc='lower left')
    plt.ylim([0, 1.2])
    plt.xlabel('Regularization parameter')
    plt.ylabel('error')
    plt.show()

we can test such function using 1 and 0 as such

In [14]:
test_svm(1, 0)

KeyboardInterrupt: 

## 2.d

the parameters W for the svm are the following


In [ ]:
clf = svm.SVC(C=2**-15, kernel='linear')
clf.fit(training_data, training_target)
print(clf.coef_)    